In [1]:
import pandas as pd
#Google colab does not have pickle
import pickle5 as pickle
import os
import seaborn as sns

import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, GlobalMaxPooling1D,Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding,Concatenate
from keras.models import Model,load_model
from sklearn.metrics import roc_auc_score,confusion_matrix,roc_curve, auc
from numpy import random
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from tensorflow.keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.metrics import AUC

import sys
import functions as f

In [2]:
five_trains,five_tests = f.load('5Fcrossval_data')


In [ ]:
EMBEDDING_DIM_5D = 25
BATCH_SIZE = 16
EPOCHS = 1
DROP = 0.5
MAX_SEQUENCE_LENGTH_5D_J = 2000
MAX_SEQUENCE_LENGTH_5D_dIP = 1000
num_words_5D_J = 1500000
num_words_5D = 1000000
scores = []

for i in range(5):
  df_train,df_test = five_trains[i],five_tests[i]
  print("Fold",i+1)
  trains = f.preprocess(df_train,saveTokrs=False)
  tests = f.preprocess(df_test,saveTokrs=False)
  x1_join = f.transf_model(MAX_SEQUENCE_LENGTH_5D_J,num_words_5D_J,5,0.9)
  x2_join = f.transf_model(MAX_SEQUENCE_LENGTH_5D_J,num_words_5D_J,5,0.9)
  x3_join = f.transf_model(MAX_SEQUENCE_LENGTH_5D_J,num_words_5D_J,5,0.9)

  x1_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
  x2_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
  x3_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
  x4_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
  x5_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
  x6_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)

  concatenator = Concatenate(axis=1)
  x = concatenator([x1_join.output, x2_join.output, x3_join.output, x1_doubleip.output, x2_doubleip.output, x3_doubleip.output, x4_doubleip.output, x5_doubleip.output, x6_doubleip.output])
  x = Dense(256)(x)
  x = Dropout(0.5)(x)
  output = Dense(1, activation="sigmoid",name="Final")(x)
  model_final = Model(inputs=[x1_join.input, x2_join.input, x3_join.input, x1_doubleip.input, x2_doubleip.input, x3_doubleip.input, x4_doubleip.input, x5_doubleip.input, x6_doubleip.input], outputs=output)

  model_final.compile(loss='binary_crossentropy', optimizer='adam', metrics=AUC())

  model_final.fit(trains, df_train['label'].values, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(tests,df_test['label'].values))
  score = roc_auc_score(df_test['label'].values, model_final.predict(tests))
  score = round(score,3)
  scores.append(score)
  #print("Score is",score)

print("AUC by cross validation is",round(sum(scores) / len(scores),2))